In [1]:
# import library
import shapefile
import rasterio
import cv2
from scipy import spatial
from scipy import ndimage as ndi
from skimage import io,filters,util,color,img_as_ubyte
from skimage.transform import hough_line, hough_line_peaks
from skimage.morphology import disk, dilation, erosion, remove_small_objects
from skimage.measure import regionprops
from skimage.feature import greycomatrix,greycoprops
import math
import numpy as np
import os
import pandas as pd

In [12]:
# Height of canopy
def plot_height(chm_path, RTK_information_path, mask_path):
    mask_line = io.imread(mask_path)
    selem = disk(7)
    Grid_Image = dilation(mask_line, selem)
    Grid_Segment = np.logical_not(Grid_Image) 
    Grid_Segment_Refine = erosion(Grid_Segment, disk(3))
    list_bbox_area = []
    Labelled_Plot_Img_o, num_features_o = ndi.measurements.label(Grid_Segment_Refine)
    for region in regionprops(Labelled_Plot_Img_o):
        list_bbox_area.append(region.area)
    re_area = np.max(list_bbox_area)*0.5
    Grid_Segment_Refine = remove_small_objects(Grid_Segment_Refine, re_area)
    Labelled_Plot_Img, num_features = ndi.measurements.label(Grid_Segment_Refine)
    
    test_dir=os.path.abspath(os.path.dirname(chm_path))
    test_dir=os.path.abspath(os.path.dirname(test_dir))
    try:
        os.mkdir(test_dir+'\\'+'process') #Create an intermediate process folder
        pro_dir=test_dir+'\\'+'process'

        os.mkdir(test_dir+'\\'+'mask') # Create the mask folder
        mask_dir=test_dir+'\\'+'mask'

        os.mkdir(test_dir+'\\'+'result') # Create the results folder
        result_dir = test_dir+'\\'+'result'
    except FileExistsError:
        pro_dir=test_dir+'\\'+'process'
        mask_dir=test_dir+'\\'+'mask'
        result_dir = test_dir+'/'+'result'
        
    sf = shapefile.Reader(RTK_information_path,'rb')
    shapes = sf.shapes()
    list_p = []
    num=0
    for i in shapes:
        list_p.append(shapes[num].points[0])
        num+=1
        
    dataset = rasterio.open(chm_path)
    upper_left_x, upper_left_y = (list_p[0][0],list_p[0][1])
    upper_left_row, upper_left_col = dataset.index(upper_left_x, upper_left_y)
    lower_left_x, lower_left_y = (list_p[1][0],list_p[1][1])
    lower_left_row, lower_left_col = dataset.index(lower_left_x, lower_left_y)
    upper_right_x, upper_right_y = (list_p[3][0],list_p[3][1])
    upper_right_row, upper_right_col = dataset.index(upper_right_x, upper_right_y)
    lower_right_x, lower_right_y = (list_p[2][0],list_p[2][1])
    lower_right_row, lower_right_col = dataset.index(lower_right_x, lower_right_y)
    
    columns_1 = np.int64(math.sqrt((upper_left_row-lower_left_row)**2
                                  + (upper_left_col-lower_left_col)**2))
    columns_2 = np.int64(math.sqrt((upper_right_row-lower_right_row)**2
                                  + (upper_right_col-lower_right_col)**2))
    columns_val = np.max((columns_1,columns_2))

    rows_1 = np.int64(math.sqrt((upper_left_row-upper_right_row)**2
                               + (upper_left_col-upper_right_col)**2))
    rows_2 = np.int64(math.sqrt((lower_left_row-lower_right_row)**2
                               + (lower_left_col-lower_right_col)**2))
    rows_val = np.max((rows_1,rows_2))
    
    img_tif = io.imread(chm_path)
    pts1 = np.float32([[upper_left_col,upper_left_row],[upper_right_col,upper_right_row],[lower_left_col,lower_left_row],
                       [lower_right_col,lower_right_row]])
    pts2 = np.float32([[0,0],[rows_val,0],[0,columns_val],[rows_val,columns_val]])
    M_PT = cv2.getPerspectiveTransform(pts1,pts2)
    dst_PT = cv2.warpPerspective(img_tif,M_PT,(rows_val,columns_val))
    
    # Extract the height
    list_cencoor = list()
    Centroid_coordinates = list()
    height_list_max = list()
    height_list_mean = list()
    height_list_median = list()
    height_list_1 = list()
    height_list_3 = list()
    height_list_5 = list()
    height_list_10 = list()
    canopy_list= list()
    for region in regionprops(Labelled_Plot_Img):
        minr, minc, maxr, maxc = region.bbox
        list_cencoor.append(region.centroid[1])
        Centroid_coordinates.append(region.centroid)
        roi=dst_PT[int(region.centroid[0])-15:int(region.centroid[0])+15,
                   int(region.centroid[1])-25:int(region.centroid[1])+25]
        roi = roi[roi>=0]
        roi = roi[roi<=3]
        height_list_max.append(np.max(roi)*100)
        height_list_mean.append(np.mean(roi)*100)
        height_list_median.append(np.median(roi)*100)
        height_list_1.append(np.mean(roi[roi>np.percentile(roi, 99)])*100)
        height_list_3.append(np.mean(roi[roi>np.percentile(roi, 97)])*100)
        height_list_5.append(np.mean(roi[roi>np.percentile(roi, 95)])*100)
        height_list_10.append(np.mean(roi[roi>np.percentile(roi, 90)])*100)
    rows = np.unique(np.array(Centroid_coordinates)[:,0]).shape[0]
    columns = int(len(Centroid_coordinates)/rows)
    row_index=[]
    column_index=[]
    for i in range(int(rows)):
        for j in range(int(columns)):
            row_index.append(i+1)
            column_index.append(j+1)
    Centroid_coordinates_y_array = np.array(list_cencoor)
    height_max_array = np.array(height_list_max)
    height_mean_array = np.array(height_list_mean)
    height_median_array = np.array(height_list_median)
    height_1_array = np.array(height_list_1)
    height_3_array = np.array(height_list_3)
    height_5_array = np.array(height_list_5)
    height_10_array = np.array(height_list_10)        
    height_max_new = []
    height_mean_new = []
    height_median_new= []
    height_1_new = []
    height_3_new = []
    height_5_new = []
    height_10_new = []
    for n in range(0, rows):
        sub_array_y = Centroid_coordinates_y_array[n*columns:columns+n*columns]
        sub_array_max_heights = height_max_array[n*columns:columns+n*columns]
        sub_array_mean_heights = height_mean_array[n*columns:columns+n*columns]
        sub_array_median_heights = height_median_array[n*columns:columns+n*columns]
        sub_array_1_height = height_1_array[n*columns:columns+n*columns]
        sub_array_3_height = height_3_array[n*columns:columns+n*columns]
        sub_array_5_height = height_5_array[n*columns:columns+n*columns]
        sub_array_10_height = height_10_array[n*columns:columns+n*columns]
        rank = np.argsort(sub_array_y)
        sub_array_max_heights = sub_array_max_heights[rank]
        sub_array_mean_heights = sub_array_mean_heights[rank]
        sub_array_median_heights = sub_array_median_heights[rank]
        sub_array_1_height = sub_array_1_height[rank]
        sub_array_3_height = sub_array_3_height[rank]
        sub_array_5_height = sub_array_5_height[rank]
        sub_array_10_height = sub_array_10_height[rank]
        for i in range(0,sub_array_max_heights.shape[0]):
            sub_element_max = sub_array_max_heights[i]
            sub_element_mean = sub_array_mean_heights[i]
            sub_element_median = sub_array_median_heights[i]
            sub_element_1 = sub_array_1_height[i]
            sub_element_3 = sub_array_3_height[i]
            sub_element_5 = sub_array_5_height[i]
            sub_element_10 = sub_array_10_height[i]

            height_max_new.append(sub_element_max)
            height_mean_new.append(sub_element_mean)
            height_median_new.append(sub_element_median)
            height_1_new.append(sub_element_1)
            height_3_new.append(sub_element_3)
            height_5_new.append(sub_element_5)
            height_10_new.append(sub_element_10)

    dt = pd.DataFrame({'rows':row_index, 'columns':column_index,'max':height_max_new,'mean':height_mean_new,'median':height_median_new,
               '1%':height_1_new,'3%':height_3_new,'5%':height_5_new,'10%':height_10_new})
    dt.to_csv(result_dir+'\\'+'_canopy_height'+'.csv',encoding="gbk")    
    display(dt)
    
    return dst_PT,rows_val,columns_val,Labelled_Plot_Img,result_dir

In [13]:
# ASM is extracted from CHM
def plot_ASM(dst_PT, Labelled_Plot_Img,result_dir):
    
    # Extract the ASM
    img_asm = dst_PT.astype(np.float64) / 255.0
    asms=[]
    list_cencoor = list()
    Centroid_coordinates = list()    
    for region in regionprops(Labelled_Plot_Img):
        minr, minc, maxr, maxc = region.bbox
        list_cencoor.append(region.centroid[1])
        Centroid_coordinates.append(region.centroid)       
        roi_all = img_asm[minr:maxr,minc:maxc]
        glcm = greycomatrix(img_as_ubyte(roi_all),[10],[0, np.pi/2], normed=True, symmetric=True)
        asm = greycoprops(glcm, 'ASM')[0]
        asms.append(np.mean(asm))        
    rows = np.unique(np.array(Centroid_coordinates)[:,0]).shape[0]
    columns = int(len(Centroid_coordinates)/rows)
    row_index=[]
    column_index=[]
    for i in range(int(rows)):
        for j in range(int(columns)):
            row_index.append(i+1)
            column_index.append(j+1)            
    Centroid_coordinates_y_array = np.array(list_cencoor)
    asms_array = np.array(asms)
    asms_new = []    
    for n in range(0, rows):
        sub_array_y = Centroid_coordinates_y_array[n*columns:columns+n*columns]
        sub_array_asms = asms_array[n*columns:columns+n*columns]
        rank = np.argsort(sub_array_y)
        sub_array_asms = sub_array_asms[rank]
        for i in range(0,sub_array_asms.shape[0]):
            sub_element_asms = sub_array_asms[i]
            asms_new.append(sub_element_asms)
    dt= pd.DataFrame({'rows':row_index, 'columns':column_index,'ASM':asms_new})
    dt.to_csv(result_dir+'\\'+'_canopy_ASM'+'.csv',encoding="gbk") 
    display(dt)
    
    return

In [14]:
# Canopy coverage
def plot_coverage(image_path, Labelled_Plot_Img,rows_val,columns_val,result_dir):
    sf = shapefile.Reader(RTK_information_path,'rb')
    shapes = sf.shapes()
    list_p = []
    num=0
    for i in shapes:
        list_p.append(shapes[num].points[0])
        num+=1
    dataset = rasterio.open(image_path)

    upper_left_x, upper_left_y = (list_p[0][0],list_p[0][1])
    upper_left_row, upper_left_col = dataset.index(upper_left_x, upper_left_y)

    lower_left_x, lower_left_y = (list_p[1][0],list_p[1][1])
    lower_left_row, lower_left_col = dataset.index(lower_left_x, lower_left_y)

    upper_right_x, upper_right_y = (list_p[3][0],list_p[3][1])
    upper_right_row, upper_right_col = dataset.index(upper_right_x, upper_right_y)

    lower_right_x, lower_right_y = (list_p[2][0],list_p[2][1])
    lower_right_row, lower_right_col = dataset.index(lower_right_x, lower_right_y)

    img_tif = io.imread(image_path)
    pts1 = np.float32([[upper_left_col,upper_left_row],[upper_right_col,upper_right_row],
                       [lower_left_col,lower_left_row],[lower_right_col,lower_right_row]])
    pts2 = np.float32([[0,0],[rows_val,0],[0,columns_val],[rows_val,columns_val]])
    M_PT = cv2.getPerspectiveTransform(pts1,pts2)
    dst_PT = cv2.warpPerspective(img_tif,M_PT,(rows_val,columns_val))
    
    #Calculate NVI
    img_indicies = dst_PT.astype(np.float64) / 255.0
    r, g, b = cv2.split(img_indicies[:,:,0:3])
    sum = r + g + b
    r = np.divide(r, sum)
    g = np.divide(g, sum)
    b = np.divide(b, sum)
    ex_g = 2.0 * g - r - b
    ex_g = ex_g * 255
    ex_g = ex_g.astype(np.uint8)
    ex_r = 1.4 * r - b
    ex_r = ex_r * 255
    ex_r = ex_r.astype(np.uint8)
    nvi = ex_g - ex_r
    nvi = nvi * 255
    nvi = nvi.astype(np.uint8)
    # Canopy coverage
    list_cencoor = list()
    Centroid_coordinates = list()
    canopy_list= list()
    area_list = list()    
    for region in regionprops(Labelled_Plot_Img):
        minr, minc, maxr, maxc = region.bbox
        list_cencoor.append(region.centroid[1])
        Centroid_coordinates.append(region.centroid)
        
        roi_all=dst_PT[minr:maxr,minc:maxc]
        roi_a=nvi[minr:maxr,minc:maxc]
        thresh = filters.threshold_otsu(roi_a)
        dst_th =(roi_a <= thresh)
        roi=roi_all.flatten()
        num_all = len(roi)
        num_canopy=len(dst_th[dst_th==0])
        canopy_cover = num_canopy/num_all
        canopy_list.append(canopy_cover)        
    rows = np.unique(np.array(Centroid_coordinates)[:,0]).shape[0]
    columns = int(len(Centroid_coordinates)/rows)
    row_index=[]
    column_index=[]
    for i in range(int(rows)):
        for j in range(int(columns)):
            row_index.append(i+1)
            column_index.append(j+1)
    Centroid_coordinates_y_array = np.array(list_cencoor)
    canopy_array = np.array(canopy_list)
    canopy_new = []   
    for n in range(0, rows):
        sub_array_y = Centroid_coordinates_y_array[n*columns:columns+n*columns]
        sub_array_canopy = canopy_array[n*columns:columns+n*columns]
        rank = np.argsort(sub_array_y)
        sub_array_canopy = canopy_array[rank]
        for i in range(0,sub_array_canopy.shape[0]):
            sub_element_canopy = sub_array_canopy[i]
            canopy_new.append(sub_element_canopy)            
    dt = pd.DataFrame({'rows':row_index, 'columns':column_index,'canopy_coverage':canopy_new})
    dt.to_csv(result_dir+'\\'+'_canopy_coverage'+'.csv',encoding="gbk")
    display(dt)
    
    return

In [15]:
# vegetation index
def plot_index(image_path, Labelled_Plot_Img,rows_val,columns_val):
    sf = shapefile.Reader(RTK_information_path,'rb')
    shapes = sf.shapes()
    list_p = []
    num=0
    for i in shapes:
        list_p.append(shapes[num].points[0])
        num+=1
    dataset = rasterio.open(image_path)

    upper_left_x, upper_left_y = (list_p[0][0],list_p[0][1])
    upper_left_row, upper_left_col = dataset.index(upper_left_x, upper_left_y)

    lower_left_x, lower_left_y = (list_p[1][0],list_p[1][1])
    lower_left_row, lower_left_col = dataset.index(lower_left_x, lower_left_y)

    upper_right_x, upper_right_y = (list_p[3][0],list_p[3][1])
    upper_right_row, upper_right_col = dataset.index(upper_right_x, upper_right_y)

    lower_right_x, lower_right_y = (list_p[2][0],list_p[2][1])
    lower_right_row, lower_right_col = dataset.index(lower_right_x, lower_right_y)

    img_tif = io.imread(image_path)
    pts1 = np.float32([[upper_left_col,upper_left_row],[upper_right_col,upper_right_row],
                       [lower_left_col,lower_left_row],[lower_right_col,lower_right_row]])
    pts2 = np.float32([[0,0],[rows_val,0],[0,columns_val],[rows_val,columns_val]])
    M_PT = cv2.getPerspectiveTransform(pts1,pts2)
    dst_PT = cv2.warpPerspective(img_tif,M_PT,(rows_val,columns_val))

    #vegetation index
    img_indicies = dst_PT.astype(np.float64) / 255.0
    r, g, b = cv2.split(img_indicies[:,:,0:3])
    sum = r + g + b
    r = np.divide(r, sum)
    g = np.divide(g, sum)
    b = np.divide(b, sum)
    vari = (g-r)/(g+r-b)
    ndyi = (g-b)/(g+b)
    img_g=color.rgb2gray(dst_PT)
    Centroid_coordinates = list()
    list_cencoor = list()
    vari_mean = list()
    ndyi_mean = list()   
    for region in regionprops(Labelled_Plot_Img):
        minr, minc, maxr, maxc = region.bbox
        list_cencoor.append(region.centroid[1])
        Centroid_coordinates.append(region.centroid)
        roi_vari = vari[int(region.centroid[0])-15:int(region.centroid[0])+15,
               int(region.centroid[1])-25:int(region.centroid[1])+25]
        roi_ndyi = ndyi[int(region.centroid[0])-15:int(region.centroid[0])+15,
               int(region.centroid[1])-25:int(region.centroid[1])+25]
        roi_ndyi = roi_ndyi[roi_ndyi>np.percentile(roi_ndyi,50)]
        roi_vari = roi_vari[roi_vari>np.percentile(roi_vari,50)]           
        ndyi_mean.append(np.mean(roi_ndyi))
        vari_mean.append(np.mean(roi_vari))
        
    rows = np.unique(np.array(Centroid_coordinates)[:,0]).shape[0]
    columns = int(len(Centroid_coordinates)/rows)

    row_index=[]
    column_index=[]
    for i in range(int(rows)):
        for j in range(int(columns)):
            row_index.append(i+1)
            column_index.append(j+1)

    Centroid_coordinates_y_array = np.array(list_cencoor)

    ndyi_array = np.array(ndyi_mean)
    vari_array = np.array(vari_mean)
    ndyi_new = []
    vari_new = []
    
    for n in range(0, rows):
        sub_array_y = Centroid_coordinates_y_array[n*columns:columns+n*columns]
        sub_array_ndyi = ndyi_array[n*columns:columns+n*columns]
        sub_array_vari = vari_array[n*columns:columns+n*columns]

        rank = np.argsort(sub_array_y)
        sub_array_ndyi = sub_array_ndyi[rank]
        sub_array_vari = sub_array_vari[rank]

        for i in range(0,sub_array_ndyi.shape[0]):
            sub_element_ndyi = sub_array_ndyi[i]
            sub_element_vari = sub_array_vari[i]

            ndyi_new.append(sub_element_ndyi)
            vari_new.append(sub_element_vari)

    dt = pd.DataFrame({'rows':row_index, 'columns':column_index,'vari':vari_new,'ndyi':ndyi_new})
    dt.to_csv(result_dir+'\\'+'_canopy_index'+'.csv',encoding="gbk")
    display(dt)
    
    img_g=color.rgb2gray(dst_PT)
    
    return img_g

In [16]:
# GLCM was extracted from orthomosaic
def plot_glcm(img_g, Labelled_Plot_Img):
    
    # Texture contrast and dissimilarity
    Centroid_coordinates = list()
    list_cencoor = list()
    glcmco = []
    glcmds = []

    for region in regionprops(Labelled_Plot_Img):
        minr, minc, maxr, maxc = region.bbox

        list_cencoor.append(region.centroid[1])
        Centroid_coordinates.append(region.centroid)
        
        roi = img_g[minr:maxr,minc:maxc]
        glcm = greycomatrix(img_as_ubyte(roi),[10],[0, np.pi/2], normed=True, symmetric=True)
        contrast = greycoprops(glcm, 'contrast')[0][0]
        glcmco.append(np.mean(contrast))
        dissimilarity = greycoprops(glcm, 'dissimilarity')[0]
        glcmds.append(np.mean(dissimilarity))

    rows = np.unique(np.array(Centroid_coordinates)[:,0]).shape[0]
    columns = int(len(Centroid_coordinates)/rows)

    row_index=[]
    column_index=[]
    for i in range(int(rows)):
        for j in range(int(columns)):
            row_index.append(i+1)
            column_index.append(j+1)

    Centroid_coordinates_y_array = np.array(list_cencoor)
    
    glcmco_array = np.array(glcmco)
    glcmds_array = np.array(glcmds)
    glcmco_new = []
    glcmds_new = []
    
    for n in range(0, rows):
        sub_array_y = Centroid_coordinates_y_array[n*columns:columns+n*columns]
        sub_array_glcmco = glcmco_array[n*columns:columns+n*columns]
        sub_array_glcmds = glcmds_array[n*columns:columns+n*columns]

        rank = np.argsort(sub_array_y)
        sub_array_glcmco = sub_array_glcmco[rank]
        sub_array_glcmds = sub_array_glcmds[rank]

        for i in range(0,sub_array_glcmds.shape[0]):
            sub_element_glcmco = sub_array_glcmco[i]
            sub_element_glcmds = sub_array_glcmds[i]

            glcmco_new.append(sub_element_glcmco)
            glcmds_new.append(sub_element_glcmds)
            
    dt = pd.DataFrame({'rows':row_index, 'columns':column_index,'contrast':glcmco_new,'dissimilarity': glcmds_new})
    dt.to_csv(result_dir+'\\'+'_canopy_glcm'+'.csv',encoding="gbk")
    display(dt)
    return

In [17]:
# Step 1: canopy height of the community
chm_path = r"C:\Users\Think\Desktop\test\3D_point_cloud\chm20190807.tif"
RTK_information_path = r"C:\Users\Think\Desktop\test\RTK_information\s_mtps.shp"
mask_path = r"C:\Users\Think\Desktop\test\mask\_mask.png"
dst_PT,rows_val,columns_val,Labelled_Plot_Img,result_dir=plot_height(chm_path, RTK_information_path, mask_path)

,rows,columns,max,mean,median,1%,3%,5%,10%
0,1,1,38.260612,20.846185,22.481185,36.420384,34.448966,33.601794,32.375738
1,1,2,25.752386,11.707580,13.027391,25.068492,23.616265,22.859657,21.654968
2,1,3,37.301230,11.044358,10.877223,31.795689,28.344160,26.129550,23.286255
3,1,4,22.881457,6.164786,6.074335,16.640152,14.789975,13.809174,12.606521
4,1,5,33.264327,9.174133,8.011951,31.424248,28.193027,25.870726,22.888473
...,...,...,...,...,...,...,...,...,...
265,18,11,30.066225,4.428517,2.392087,24.895431,21.051563,19.141199,16.432291
266,18,12,33.327496,6.210013,4.617633,25.993326,22.217143,20.520654,18.133989
267,18,13,27.017641,5.912658,3.114705,25.463775,23.846233,22.888930,20.830385
268,18,14,23.134553,1.710819,0.611219,17.984647,14.065813,11.422689,8.585912


In [18]:
#Step 2: Extract ASM from CHM
plot_ASM(dst_PT, Labelled_Plot_Img,result_dir)

,rows,columns,ASM
0,1,1,1.0
1,1,2,1.0
2,1,3,1.0
3,1,4,1.0
4,1,5,1.0
...,...,...,...
265,18,11,1.0
266,18,12,1.0
267,18,13,1.0
268,18,14,1.0


In [19]:
# Step 3: Canopy coverage
plot_coverage(dst_PT, Labelled_Plot_Img,result_dir)

,rows,columns,canopy_coverage
0,1,1,0.332000
1,1,2,0.130667
2,1,3,0.039333
3,1,4,0.008000
4,1,5,0.038667
...,...,...,...
265,18,11,0.330667
266,18,12,0.115333
267,18,13,0.604000
268,18,14,0.346667


In [20]:
# Step 4: Vegetation Index
image_path=r"C:\Users\Think\Desktop\test\2D_orthomosaic_image\20190807.tif"
img_g=plot_index(image_path, Labelled_Plot_Img,rows_val,columns_val)

,rows,columns,vari,ndyi
0,1,1,0.370962,0.395451
1,1,2,0.291365,0.391704
2,1,3,0.281201,0.390063
3,1,4,0.311164,0.402681
4,1,5,0.314976,0.435065
...,...,...,...,...
265,18,11,0.202198,0.416153
266,18,12,0.247110,0.350073
267,18,13,0.225138,0.229859
268,18,14,0.249665,0.269039


In [21]:
# Step 5: Extraction of GLCM from orthomosaic
plot_glcm(img_g, Labelled_Plot_Img)

,rows,columns,contrast,dissimilarity
0,1,1,1128.604747,29.906616
1,1,2,817.849390,27.440347
2,1,3,913.985843,27.762064
3,1,4,930.096768,31.982259
4,1,5,1338.213963,32.905857
...,...,...,...,...
265,18,11,1869.744391,36.073738
266,18,12,2192.382822,38.266436
267,18,13,1855.930312,32.667543
268,18,14,2057.289491,37.518244
